In [1]:
import os
import time
from dotenv import load_dotenv
import pandas as pd
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials

from tqdm import tqdm
from spotipy import Spotify
from pandas import DataFrame


In [2]:
client_id = '5ab3d5c1f176456788d74a39134a4c23'
client_secret = 'ed854e82eb764dc8ac60331cc7afdf65'

In [3]:
# Spotify API authentication
client_credentials_manager = SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret
)
sp = spotipy.Spotify(
    client_credentials_manager=client_credentials_manager
)

In [4]:
#usuario
user = 'lucas-oli'
playlist_test = '6nsNG25T4kYCtFrywWZr1k'

In [5]:
#Get several tracks from a playlist using spotipy

def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [6]:
tracks = get_playlist_tracks(user,playlist_test)

In [7]:

def get_track_info(track):
    return {
        "id": track["id"],
        "name": track["name"],
        "album_id": track["album"]["id"],
    }

#Get several trakc sfrom a specif playlist
def get_tracks_from_playlist(playlist_id, sp):
    playlist_tracks = sp.playlist_tracks(playlist_id=playlist_id)
    tracks_playlist = [get_track_info(item["track"]) for item in playlist_tracks["items"]]
    return tracks_playlist

In [8]:
df_tracks = pd.DataFrame(get_tracks_from_playlist(playlist_test, sp=sp))

In [9]:
df_tracks.head()

,id,name,album_id
0,1M4Wy6jRce1x17qyE7yQAB,The Feast and The Famine,2qwN15acAl3sm3Idce5vK9
1,0SmyaVrzL7j3R9k3pkJgUh,Dummy – Chris Lake Remix,6XqbzujsQpFewramig6kUM
2,6jHvX8ZnHKC1PnrPMJ0Emt,Cigarette Daydreams,5nJ9B1KiydXoYWAcyqlG76
3,4waqcUQWdj0yH26STWl2Rq,Rosas,1Atbkyir7YdfpFYWCnRTFp
4,57Xjny5yNzAcsxnusKmAfA,Reptilia,3HFbH1loOUbqCyPsLuHLLh


In [10]:
# Get audio features from tracks
audio_features = []

for n in tqdm(range(len(df_tracks) // 100 + 1)):
    df_track_pack = df_tracks.iloc[n*100:(n+1)*100]
    audio_features_pack = sp.audio_features(tracks=list(df_track_pack['id']))
    audio_features += audio_features_pack

df_audio_features = pd.DataFrame(audio_features)
print('Shape audio features:', df_audio_features.shape)

100%|██████████| 1/1 [00:00<00:00,  2.61it/s]

Shape audio features: (22, 18)


In [11]:
df_audio_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.332,0.906,11,-4.683,0,0.1050,0.000056,0.000304,0.1290,0.497,173.639,audio_features,1M4Wy6jRce1x17qyE7yQAB,spotify:track:1M4Wy6jRce1x17qyE7yQAB,https://api.spotify.com/v1/tracks/1M4Wy6jRce1x...,https://api.spotify.com/v1/audio-analysis/1M4W...,229560,4
1,0.789,0.867,6,-6.925,1,0.0539,0.000608,0.691000,0.0733,0.699,127.016,audio_features,0SmyaVrzL7j3R9k3pkJgUh,spotify:track:0SmyaVrzL7j3R9k3pkJgUh,https://api.spotify.com/v1/tracks/0SmyaVrzL7j3...,https://api.spotify.com/v1/audio-analysis/0Smy...,172937,4
2,0.636,0.676,2,-3.442,1,0.0263,0.080700,0.000000,0.0831,0.273,113.980,audio_features,6jHvX8ZnHKC1PnrPMJ0Emt,spotify:track:6jHvX8ZnHKC1PnrPMJ0Emt,https://api.spotify.com/v1/tracks/6jHvX8ZnHKC1...,https://api.spotify.com/v1/audio-analysis/6jHv...,208760,4
3,0.783,0.685,9,-5.008,1,0.0271,0.627000,0.000003,0.1080,0.540,107.010,audio_features,4waqcUQWdj0yH26STWl2Rq,spotify:track:4waqcUQWdj0yH26STWl2Rq,https://api.spotify.com/v1/tracks/4waqcUQWdj0y...,https://api.spotify.com/v1/audio-analysis/4waq...,236333,4
4,0.489,0.649,9,-5.110,1,0.0336,0.000603,0.713000,0.1010,0.770,158.009,audio_features,57Xjny5yNzAcsxnusKmAfA,spotify:track:57Xjny5yNzAcsxnusKmAfA,https://api.spotify.com/v1/tracks/57Xjny5yNzAc...,https://api.spotify.com/v1/audio-analysis/57Xj...,219827,4


In [12]:
df_audio_features.shape

(22, 18)

In [13]:
# Merge dataframes df_tracks and df_audio_features
df_tracks_audio_features = pd.merge(
    df_tracks,
    df_audio_features,
    on="id",
    how="inner"
)

In [14]:
df_tracks_audio_features.head()

,id,name,album_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,1M4Wy6jRce1x17qyE7yQAB,The Feast and The Famine,2qwN15acAl3sm3Idce5vK9,0.332,0.906,11,-4.683,0,0.1050,0.000056,0.000304,0.1290,0.497,173.639,audio_features,spotify:track:1M4Wy6jRce1x17qyE7yQAB,https://api.spotify.com/v1/tracks/1M4Wy6jRce1x...,https://api.spotify.com/v1/audio-analysis/1M4W...,229560,4
1,0SmyaVrzL7j3R9k3pkJgUh,Dummy – Chris Lake Remix,6XqbzujsQpFewramig6kUM,0.789,0.867,6,-6.925,1,0.0539,0.000608,0.691000,0.0733,0.699,127.016,audio_features,spotify:track:0SmyaVrzL7j3R9k3pkJgUh,https://api.spotify.com/v1/tracks/0SmyaVrzL7j3...,https://api.spotify.com/v1/audio-analysis/0Smy...,172937,4
2,6jHvX8ZnHKC1PnrPMJ0Emt,Cigarette Daydreams,5nJ9B1KiydXoYWAcyqlG76,0.636,0.676,2,-3.442,1,0.0263,0.080700,0.000000,0.0831,0.273,113.980,audio_features,spotify:track:6jHvX8ZnHKC1PnrPMJ0Emt,https://api.spotify.com/v1/tracks/6jHvX8ZnHKC1...,https://api.spotify.com/v1/audio-analysis/6jHv...,208760,4
3,4waqcUQWdj0yH26STWl2Rq,Rosas,1Atbkyir7YdfpFYWCnRTFp,0.783,0.685,9,-5.008,1,0.0271,0.627000,0.000003,0.1080,0.540,107.010,audio_features,spotify:track:4waqcUQWdj0yH26STWl2Rq,https://api.spotify.com/v1/tracks/4waqcUQWdj0y...,https://api.spotify.com/v1/audio-analysis/4waq...,236333,4
4,57Xjny5yNzAcsxnusKmAfA,Reptilia,3HFbH1loOUbqCyPsLuHLLh,0.489,0.649,9,-5.110,1,0.0336,0.000603,0.713000,0.1010,0.770,158.009,audio_features,spotify:track:57Xjny5yNzAcsxnusKmAfA,https://api.spotify.com/v1/tracks/57Xjny5yNzAc...,https://api.spotify.com/v1/audio-analysis/57Xj...,219827,4


In [15]:
df_tracks_audio_features.shape

(22, 20)

In [16]:
# Save df_audio_features to csv
df_tracks_audio_features.to_csv('./datasets/test_final/tracks_audio_features.csv', index=False)

In [17]:
df_tracks_audio_features.shape

(22, 20)

In [18]:
df_tracks_audio_features = pd.read_csv('./datasets/test_final/tracks_audio_features.csv')

In [19]:
df_tracks_audio_features.columns

Index(['id', 'name', 'album_id', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'uri', 'track_href', 'analysis_url',
       'duration_ms', 'time_signature'],
      dtype='object')

In [20]:
df_tracks_audio_features['key']=df_tracks_audio_features['key'].astype('category')
df_tracks_audio_features['time_signature']=df_tracks_audio_features['time_signature'].astype('category')
df_tracks_audio_features['mode']=df_tracks_audio_features['mode'].astype('category')


In [21]:
included_types = ['float64', 'int64','category']
selected_columns = df_tracks_audio_features.select_dtypes(include=included_types).columns
selected_variables = df_tracks_audio_features[selected_columns]

In [22]:
selected_variables.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.332,0.906,11,-4.683,0,0.1050,0.000056,0.000304,0.1290,0.497,173.639,229560,4
1,0.789,0.867,6,-6.925,1,0.0539,0.000608,0.691000,0.0733,0.699,127.016,172937,4
2,0.636,0.676,2,-3.442,1,0.0263,0.080700,0.000000,0.0831,0.273,113.980,208760,4
3,0.783,0.685,9,-5.008,1,0.0271,0.627000,0.000003,0.1080,0.540,107.010,236333,4
4,0.489,0.649,9,-5.110,1,0.0336,0.000603,0.713000,0.1010,0.770,158.009,219827,4


In [23]:
selected_variables.shape

(22, 13)

In [24]:
selected_variables.to_csv('./datasets/test_final/selected_variables.csv', index=False)

## Extramos data from this is foogifhters and this is trueno

In [41]:
this_is_trueno = '37i9dQZF1DZ06evO1s4mic'
this_is_foo = '37i9dQZF1DZ06evO4kqwHC'

In [42]:
df_this_is_trueno = pd.DataFrame(get_tracks_from_playlist(this_is_trueno, sp=sp))
df_this_is_foo = pd.DataFrame(get_tracks_from_playlist(this_is_foo, sp=sp))

In [43]:
df_this_is_trueno.head()

,id,name,album_id
0,3R8XAtUiwnjePpeJ6SghO6,DUBAI,6LYXmkU5doOyO7XQ3yFLTd
1,0TUW9faHNaBmi89wsYGp9y,Mamichula - con Nicki Nicole,1xBoZOfcOsqd77V6AENKYC
2,3qukYaYSxIUBcq5UIUIsPf,FEEL ME??,1HeNYlqvbUDkP97DJ33Kjl
3,3UytZp1xDkNkROyIwE7DbY,Hood - Remix,4ojUS0aIHkPPxworw8ZMTv
4,1EljzqiV1RDtXj1QUbx3db,Rain,2rmhl1gQux3WGPJA77FgRN


In [44]:
df_this_is_foo.head()

,id,name,album_id
0,26iWAXoJRiUv9yYzxYBqSv,The Glass,4wp4aWWpoYfNcspimVAnel
1,5UWwZ5lm5PKu6eKsHAGxOk,Everlong,30ly6F6Xl0TKmyBCU50Khv
2,5OQsiBsky2k2kDKy2bX2eT,Learn to Fly,28q2N44ocJECgf8sbHEDfY
3,7x8dCjCr0x6x2lXKujYD34,The Pretender,3ilXDEG0xiajK8AbqboeJz
4,5FZxsHWIvUsmSK1IAvm2pp,Best of You,2eprpJCYbCbPZRKVGIEJxZ


In [45]:
# Get audio features from tracks
def get_audio_features(df_tracks,sp):
    audio_features = []

    for n in tqdm(range(len(df_tracks) // 100 + 1)):
        df_track_pack = df_tracks.iloc[n*100:(n+1)*100]
        audio_features_pack = sp.audio_features(tracks=list(df_track_pack['id']))
        audio_features += audio_features_pack

    return pd.DataFrame(audio_features)
    

In [46]:
df_this_is_trueno_audio_features = get_audio_features(df_this_is_trueno,sp=sp)
df_this_is_foo_audio_features = get_audio_features(df_this_is_foo,sp=sp)

100%|██████████| 1/1 [00:00<00:00,  3.19it/s]


In [47]:
# Merge dataframes df_tracks and df_audio_features
def merge_tracks_audio_features(df_tracks,df_audio_features):
    return pd.merge(
        df_tracks,
        df_audio_features,
        on="id",
        how="inner"
    )

df_this_is_trueno_tracks_audio_features = merge_tracks_audio_features(df_this_is_trueno,df_this_is_trueno_audio_features)
df_this_is_foo_tracks_audio_features = merge_tracks_audio_features(df_this_is_foo,df_this_is_foo_audio_features)

In [48]:
df_this_is_trueno_tracks_audio_features.head()

,id,name,album_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,3R8XAtUiwnjePpeJ6SghO6,DUBAI,6LYXmkU5doOyO7XQ3yFLTd,0.801,0.756,5,-1.655,0,0.0542,0.589,0.000001,0.0842,0.682,93.990,audio_features,spotify:track:3R8XAtUiwnjePpeJ6SghO6,https://api.spotify.com/v1/tracks/3R8XAtUiwnje...,https://api.spotify.com/v1/audio-analysis/3R8X...,146000,4
1,0TUW9faHNaBmi89wsYGp9y,Mamichula - con Nicki Nicole,1xBoZOfcOsqd77V6AENKYC,0.696,0.678,4,-5.893,0,0.0735,0.582,0.000002,0.1340,0.346,91.985,audio_features,spotify:track:0TUW9faHNaBmi89wsYGp9y,https://api.spotify.com/v1/tracks/0TUW9faHNaBm...,https://api.spotify.com/v1/audio-analysis/0TUW...,219202,4
2,3qukYaYSxIUBcq5UIUIsPf,FEEL ME??,1HeNYlqvbUDkP97DJ33Kjl,0.733,0.568,1,-5.987,1,0.0462,0.341,0.000000,0.2320,0.502,123.079,audio_features,spotify:track:3qukYaYSxIUBcq5UIUIsPf,https://api.spotify.com/v1/tracks/3qukYaYSxIUB...,https://api.spotify.com/v1/audio-analysis/3quk...,187332,4
3,3UytZp1xDkNkROyIwE7DbY,Hood - Remix,4ojUS0aIHkPPxworw8ZMTv,0.674,0.456,3,-6.253,1,0.0399,0.785,0.000000,0.1260,0.933,89.967,audio_features,spotify:track:3UytZp1xDkNkROyIwE7DbY,https://api.spotify.com/v1/tracks/3UytZp1xDkNk...,https://api.spotify.com/v1/audio-analysis/3Uyt...,176373,4
4,1EljzqiV1RDtXj1QUbx3db,Rain,2rmhl1gQux3WGPJA77FgRN,0.543,0.686,8,-7.848,1,0.4090,0.953,0.000052,0.6490,0.562,89.863,audio_features,spotify:track:1EljzqiV1RDtXj1QUbx3db,https://api.spotify.com/v1/tracks/1EljzqiV1RDt...,https://api.spotify.com/v1/audio-analysis/1Elj...,182831,5


In [49]:
def normalize(df_tracks):
    df_tracks['key']=df_tracks['key'].astype('category')
    df_tracks['time_signature']=df_tracks['time_signature'].astype('category')
    df_tracks['mode']=df_tracks['mode'].astype('category')
    return df_tracks


In [36]:
df_this_is_trueno_tracks_audio_features = normalize(df_this_is_trueno_tracks_audio_features)
df_this_is_foo_tracks_audio_features = normalize(df_this_is_foo_tracks_audio_features)

In [40]:
this_is_trueno = df_this_is_trueno_tracks_audio_features.to_csv('./datasets/test_final/this_is_trueno.csv')
this_is_foo = df_this_is_foo_tracks_audio_features.to_csv('./datasets/test_final/this_is_foo.csv')

In [37]:
def select (df_tracks):
    included_types = ['float64', 'int64','category']
    selected_columns = df_tracks.select_dtypes(include=included_types).columns
    selected_variables = df_tracks[selected_columns]
    return selected_variables

df_this_is_trueno_tracks_audio_features = select(df_this_is_trueno_tracks_audio_features)
df_this_is_foo_tracks_audio_features = select(df_this_is_foo_tracks_audio_features)


In [38]:
#Save to csv 

df_this_is_trueno_tracks_audio_features.to_csv('./datasets/test_final/this_is_trueno_tracks_audio_features.csv', index=False)
df_this_is_foo_tracks_audio_features.to_csv('./datasets/test_final/this_is_foo_tracks_audio_features.csv', index=False)